In [30]:
from os import getenv
from dotenv import load_dotenv
import requests as req


In [31]:
load_dotenv()

access_token = getenv('GITHUB_ACCESS_TOKEN')

headers = {'X-GitHub-Api-Version': '2022-11-28',
           'Authorization': f'Bearer {access_token}'}


# Desafio 1

Faça uma requisição GET à API do Github para obter as informações do perfil de um usuário específico. Para isso, utilize a biblioteca Requests do Python e a URL https://api.github.com/users/{username}, substituindo {username} pelo nome de usuário desejado.

Com o objeto response retornado pela requisição, imprima na tela as seguintes informações sobre a requisição:

* status code da requisição, para analisar se foi bem sucedida ou não;
* conteúdo da requisição, para conferir se os dados de usuário vieram corretamente;
* URL da requisição feita, para garantir que a url utilizada na requisição foi a correta.

Após conferir isso, analise novamente o conteúdo retornado pela requisição e imprima na tela apenas as seguintes informações sobre a pessoa usuária em questão:

* Nome
* Nome de usuário
* Número de repositórios públicos
* Data de criação da conta no GitHub


In [33]:
username = 'not-valid-user'

print('Abaixo veremos uma request que não vai dar certo por conta do user não existir.')

req.get(f'https://api.github.com/users/{username}', headers).json()


Abaixo veremos uma request que não vai dar certo por conta do user não existir.


{'message': "API rate limit exceeded for 179.130.165.183. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)",
 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}

In [11]:
username = 'juanveronez'
res = req.get(f'https://api.github.com/users/{username}')


In [12]:
print(f'Status Code: {res.status_code}')
print(f'Url: {res.url}')
print(f'\nConteúdo da requisição:\n{res.json()}')


Status Code: 403
Url: https://api.github.com/users/juanveronez

Conteúdo da requisição:
{'message': "API rate limit exceeded for 179.130.165.183. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)", 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}


In [13]:
json = res.json()
type(json)


dict

In [14]:
print(f'Nome: {json['name']}')
print(f'Nome de usuário: {json['login']}')
print(f'Número de repositórios públicos: {json['public_repos']}')
print(f'Data de criação da conta no GitHub: {json['created_at']}')


KeyError: 'name'

In [27]:
load_dotenv()

access_token = getenv('GITHUB_ACCESS_TOKEN')

headers = {'X-GitHub-Api-Version': '2022-11-28',
           'Authorization': f'Bearer {access_token}'}


In [22]:
username = 'amzn'
url = f"https://api.github.com/users/{username}/followers"

response = requests.get(url, headers)
followers = response.json()


In [23]:
followers_lists = []
while True:
    list_index = len(followers_lists) + 1
    page_url = f'{url}?per_page=100&page={list_index}'

    list = requests.get(page_url, headers).json()

    if len(list) == 0:
        break
    
    followers_lists.append(list)

followers_lists


ConnectTimeout: HTTPSConnectionPool(host='api.github.com', port=443): Max retries exceeded with url: /users/amzn/followers?per_page=100&page=283&X-GitHub-Api-Version=2022-11-28&Authorization=Bearer+None (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10f875e50>, 'Connection to api.github.com timed out. (connect timeout=None)'))